In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=1f26c8f23a0678fc8c904b9b10fe68e065d02e88236cb2e15a5ba60e3e8d52fe
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install bert-score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyli

In [ ]:
import pandas as pd
import re
import nltk
import networkx as nx
from transformers import pipeline
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# Download NLTK data
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
file_path = '/content/drive/MyDrive/CV.csv'
data = pd.read_csv(file_path)


In [ ]:
def split_into_sections(text):
    # Implement section splitting logic here (if needed)
    sections = [text]  # For simplicity, treat the entire text as one section
    return sections

def split_into_paragraphs(text):
    paragraphs = re.split(r'\n\n+', text)  # Split based on empty lines
    return paragraphs

# Initialize BERT NER pipeline outside the function to avoid redundant initialization
ner_pipeline = pipeline("ner", model="bert-base-uncased", tokenizer="bert-base-uncased")

def determine_subject_with_BERT(paragraph):
    # Perform NER on paragraph
    entities = ner_pipeline(paragraph)

    # Extract main subject (e.g., first entity)
    if entities:
        return entities[0]['word']
    else:
        return None

def extract_nodes_and_edges(paragraph):
    tokens = nltk.word_tokenize(paragraph)
    pos_tags = nltk.pos_tag(tokens)
    return pos_tags

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def build_graph(data):
    G = nx.DiGraph()
    for abstract in data['abstract']:
        sections = split_into_sections(abstract)
        for section in sections:
            paragraphs = split_into_paragraphs(section)
            for paragraph in paragraphs:
                subject = determine_subject_with_BERT(paragraph)
                if subject:
                    G.add_node(subject, pos='noun')
                    nodes_and_edges = extract_nodes_and_edges(paragraph)
                    if len(nodes_and_edges) > 0:
                        for word, pos in nodes_and_edges:
                            if pos in ['NN', 'NNS', 'JJ', 'RB']:
                                G.add_node(word, pos=pos.lower())
                                G.add_edge(subject, word, relation=pos.lower())
                        # Check for the existence of at least one word node
                        if any(word for word, _ in nodes_and_edges):
                            # Check if source and target nodes exist
                            if G.has_node(subject) and G.has_node(nodes_and_edges[0][0]):
                                if nx.has_path(G, subject, nodes_and_edges[0][0]):
                                    for word, pos in nodes_and_edges:
                                        if pos in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'RB']:
                                            G.add_edge(subject, word, relation=pos.lower())
    return G

# Build the graph
graph = build_graph(data)



In [ ]:
import pandas as pd
import re
import nltk
import networkx as nx
from transformers import pipeline
from bert_score import score as bert_score
import matplotlib.pyplot as plt

# Download NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

file_path = '/content/drive/MyDrive/CV.csv'
data = pd.read_csv(file_path)

def split_into_sections(text):
    sections = [text]  # For simplicity, treat the entire text as one section
    return sections

def split_into_paragraphs(text):
    paragraphs = re.split(r'\n\n+', text)  # Split based on empty lines
    return paragraphs

def determine_subject_with_BERT(paragraph):
    ner_pipeline = pipeline("ner", model="bert-base-uncased", tokenizer="bert-base-uncased")
    entities = ner_pipeline(paragraph)
    if entities:
        return entities[0]['word']
    else:
        return None

def extract_nodes_and_edges(paragraph):
    tokens = nltk.word_tokenize(paragraph)
    pos_tags = nltk.pos_tag(tokens)
    return pos_tags

def build_graph(data):
    G = nx.DiGraph()
    for abstract in data['abstract']:
        sections = split_into_sections(abstract)
        for section in sections:
            paragraphs = split_into_paragraphs(section)
            for paragraph in paragraphs:
                subject = determine_subject_with_BERT(paragraph)
                if subject:
                    G.add_node(subject, pos='noun')
                    nodes_and_edges = extract_nodes_and_edges(paragraph)
                    if len(nodes_and_edges) > 0:
                        for word, pos in nodes_and_edges:
                            if pos in ['NN', 'NNS', 'JJ', 'RB']:
                                G.add_node(word, pos=pos.lower())
                                G.add_edge(subject, word, relation=pos.lower())
                        # Check for the existence of at least one word node
                        if any(word for word, _ in nodes_and_edges):
                            # Check if source and target nodes exist
                            if G.has_node(subject) and G.has_node(nodes_and_edges[0][0]):
                                if nx.has_path(G, subject, nodes_and_edges[0][0]):
                                    for word, pos in nodes_and_edges:
                                        if pos in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'RB']:
                                            G.add_edge(subject, word, relation=pos.lower())
    return G

def visualize_graph(G):
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, seed=42)  # Use spring layout algorithm
    nx.draw(G, pos, with_labels=True, node_size=2000, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=20)
    plt.title("Graph Visualization")
    plt.show()

def generate_summary(abstract, graph):
    summary = []
    for sentence in nltk.sent_tokenize(abstract):
        keywords = [word.lower() for word, pos in nltk.pos_tag(nltk.word_tokenize(sentence)) if pos in ['NN', 'NNS', 'JJ', 'RB']]
        if keywords:
            main_subject = keywords[0]
            if main_subject in graph.nodes:
                summary.append(main_subject)
                related_words = find_related_words(graph, main_subject, 0, 3)
                summary.extend(related_words)
    return ' '.join(summary)

def find_related_words(graph, node, current_level, max_level):
    if current_level >= max_level:
        return []
    else:
        related_words = []
        successors = graph.successors(node)
        for successor in successors:
            relation = graph.get_edge_data(node, successor)['relation']
            related_words.append(successor)
            related_words.extend(find_related_words(graph, successor, current_level + 1, max_level))
        return related_words

# Build the graph
graph = build_graph(data)

# Visualize the graph
visualize_graph(graph)

# Generate summaries for all abstracts
summaries = []
for abstract in data['abstract']:
    summary = generate_summary(abstract, graph)
    summaries.append(summary)

# Compute BERTScore for each summary
references = data['abstract'].tolist()
bert_scores = bert_score(summaries, references, lang='en')

print("BERTScore for each summary:")
for i, score in enumerate(bert_scores):
    print(f"Summary {i+1}: {score.mean().item()}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initi

In [ ]:
import pandas as pd
import re
import nltk
import networkx as nx
from transformers import pipeline

# Download NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Initialize BERT NER pipeline
ner_pipeline = pipeline("ner", model="bert-base-uncased", tokenizer="bert-base-uncased")

file_path = '/content/drive/MyDrive/CV.csv'
data = pd.read_csv(file_path)

def split_into_sections(text):
    sections = [text]  # For simplicity, treat the entire text as one section
    return sections

def split_into_paragraphs(text):
    paragraphs = re.split(r'\n\n+', text)  # Split based on empty lines
    return paragraphs

def determine_subject_with_BERT(paragraph):
    # Perform NER on paragraph
    entities = ner_pipeline(paragraph)

    # Extract main subject (e.g., first entity)
    if entities:
        return entities[0]['word']
    else:
        return None

def extract_nodes_and_edges(paragraph):
    tokens = nltk.word_tokenize(paragraph)
    pos_tags = nltk.pos_tag(tokens)
    return pos_tags

def build_graph(data):
    G = nx.DiGraph()
    for abstract in data['abstract']:
        sections = split_into_sections(abstract)
        for section in sections:
            paragraphs = split_into_paragraphs(section)
            for paragraph in paragraphs:
                subject = determine_subject_with_BERT(paragraph)
                if subject:
                    G.add_node(subject, pos='noun')
                    nodes_and_edges = extract_nodes_and_edges(paragraph)
                    if len(nodes_and_edges) > 0:
                        for word, pos in nodes_and_edges:
                            if pos in ['NN', 'NNS', 'JJ', 'RB']:
                                G.add_node(word, pos=pos.lower())
                                G.add_edge(subject, word, relation=pos.lower())
                        if any(word for word, _ in nodes_and_edges):
                            if G.has_node(subject) and G.has_node(nodes_and_edges[0][0]):
                                if nx.has_path(G, subject, nodes_and_edges[0][0]):
                                    for word, pos in nodes_and_edges:
                                        if pos in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'RB']:
                                            G.add_edge(subject, word, relation=pos.lower())
    return G

# Build the graph
graph = build_graph(data)

import matplotlib.pyplot as plt
from bert_score import score as bert_score

# Visualize the graph
def visualize_graph(G):
    nx.draw(G, with_labels=True)
    plt.show()

visualize_graph(graph)

# Generate summaries for all abstracts
summaries = []
for abstract in data['abstract']:
    summary = generate_summary(abstract, graph)
    summaries.append(summary)

# Compute BERTScore for each summary
references = data['abstract'].tolist()
bert_scores = bert_score(summaries, references, lang='en')

print("BERTScore for each summary:")
for i, score in enumerate(bert_scores):
    print(f"Summary {i+1}: {score.mean().item()}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
